In [25]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-134526
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-134526


In [26]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
cpu_cluster_name = "cpuacluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
    
except ComputeTargetException:
    
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    print("New cluster is created")


Found existing cluster


In [27]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice,randint
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        'C':uniform(0,1),
        "max_iter":randint(150)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

#import shutil
#shutil.copy('train.py','training/')
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',  
              compute_target=cpu_cluster,
              entry_script='train.py', 
              vm_size = 'STANDARD_D2_V2', 
              vm_priority = 'lowpriority')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10,
                                     max_concurrent_runs=4)

In [28]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr=exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [29]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('parameter values:',parameter_values)
print('\n Best run \n',best_run)

print('The Id for best run is:', best_run.id)
print('The Accuracy: is', best_run_metrics['Accuracy'])


parameter values: ['--C', '0.3532829009310535', '--max_iter', '96']

 Best run 
 Run(Experiment: udacity-project1,
Id: HD_e70ea350-f4b0-4f9e-bd1a-5a8aa7b7321a_1,
Type: azureml.scriptrun,
Status: Completed)
The Id for best run is: HD_e70ea350-f4b0-4f9e-bd1a-5a8aa7b7321a_1
The Accuracy: is 0.9132018209408195


In [45]:
#get files
best_run.get_file_names()


['azureml-logs/55_azureml-execution-tvmps_ba695b49f9a286d4fe6858135c6ad96760bc6c538e9e38332cc9d99649fc82a0_d.txt',
 'azureml-logs/65_job_prep-tvmps_ba695b49f9a286d4fe6858135c6ad96760bc6c538e9e38332cc9d99649fc82a0_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_ba695b49f9a286d4fe6858135c6ad96760bc6c538e9e38332cc9d99649fc82a0_d.txt',
 'logs/azureml/95_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/engine_spans_l_19357022-8e8b-4733-8dd7-3c4ff430443a.jsonl',
 'logs/azureml/dataprep/python_span_l_19357022-8e8b-4733-8dd7-3c4ff430443a.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [47]:
#register model
the_best_model = best_run.register_model(model_name='best-model.joblib', model_path='./')

In [48]:
#Display the best model
the_best_model

Model(workspace=Workspace.create(name='quick-starts-ws-134526', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-134526'), name=best-model.joblib, id=best-model.joblib:1, version=1, tags={}, properties={})

In [49]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_path= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path = data_path)

In [50]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [51]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target = cpu_cluster,
    task="classification",
    primary_metric="accuracy",
    training_data= ds,
    label_column_name='y',
    iterations=20,
    max_concurrent_iterations=4,
    n_cross_validations=4)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

remote_run=exp.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpuacluster with default configuration
Running on remote compute: cpuacluster
Parent Run ID: AutoML_b6b9ca85-3a35-4749-9238-9dca774fc01b

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input

In [19]:
#getting the metrics required
best_run_auto, fitted_model_auto = remote_run.get_output()
best_run_metrics = best_run_auto.get_metrics()
parameter_values = best_run_auto.get_details()['runDefinition']['arguments']

print('parameter values:',parameter_values)
print('The best run automl model\n', best_run_auto)

print('\nThe Id for best run is:', best_run_auto.id)
print('The Accuracy: is', best_run_metrics['accuracy'])

parameter values: []
The best run automl model
 Run(Experiment: udacity-project1,
Id: AutoML_65c714cc-7149-4b40-86fb-deaf89fc1141_18,
Type: azureml.scriptrun,
Status: Completed)

The Id for best run is: AutoML_65c714cc-7149-4b40-86fb-deaf89fc1141_18
The Accuracy: is 0.9169042639835655


In [22]:
import joblib
# Retrieve and save your best automl model.
best_run_model, fitted_model_a = remote_run.get_output()

joblib.dump(fitted_model_a, "model.joblib")

['model.joblib']

In [24]:
print('The model is:\n',fitted_model_a)

The model is:
 Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_impurity_split=None,
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.01,
     